<div class="alert alert-danger">
    <h4 style="font-weight: bold; font-size: 28px;">Feature Engineering</h4>
    <p style="font-size: 20px;">NBA API Data (1984-2024)</p>
</div>

<a name="Feature Engineering"></a>

# Table of Contents

[Setup](#Setup)

[Data](#Data)

**[1. Create Team Matchups and Targets](#1.-Create-Team-Matchups-and-Targets)**

- [1.1. Remove any Orphan Matchups](#1.1.-Remove-any-Orphan-Matchups)

- [1.2. Reshape to Game Matchups](#1.2.-Reshape-to-Game-Matchups)

- [1.3. Create Target Variables](#1.3.-Create-Target-Variables)

**[2. Create Rolling Window Statistics](#2.-Create-Rolling-Window-Statistics)**

# Setup

[Return to top](#Feature-Engineering)

In [1]:
# basic modules
import os
import time
import random as rn
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# plotting style
plt.style.use('seaborn-v0_8-notebook')
sns.set_style('white')
#sns.set_style('darkgrid')

# pandas tricks for better display
pd.options.display.max_columns = 50  
pd.options.display.max_rows = 500     
pd.options.display.max_colwidth = 100
pd.options.display.precision = 3

# preprocessing
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin

# warnings
import warnings
warnings.filterwarnings("ignore")

# user defined functions
import utility_functions as utl

# Data

[Return to top](#Feature-Engineering)

In [2]:
team_bs_df = pd.read_csv('../data/original/nba_games_1984_2024.csv')
player_bs_df = pd.read_csv('../data/original/nba_players_statistics_1946_2024.csv')

In [3]:
# convert the 'GAME_DATE' to datetime
team_bs_df['GAME_DATE'] = pd.to_datetime(team_bs_df['GAME_DATE'])

In [4]:
team_bs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104437 entries, 0 to 104436
Data columns (total 28 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   SEASON_ID          104437 non-null  int64         
 1   TEAM_ID            104437 non-null  int64         
 2   TEAM_ABBREVIATION  104437 non-null  object        
 3   TEAM_NAME          104437 non-null  object        
 4   GAME_ID            104437 non-null  int64         
 5   GAME_DATE          104437 non-null  datetime64[ns]
 6   MATCHUP            104437 non-null  object        
 7   WL                 104427 non-null  object        
 8   MIN                104437 non-null  int64         
 9   PTS                104437 non-null  int64         
 10  FGM                104437 non-null  int64         
 11  FGA                104437 non-null  int64         
 12  FG_PCT             104430 non-null  float64       
 13  FG3M               104437 non-null  int64   

In [5]:
player_bs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29746 entries, 0 to 29745
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   PLAYER_ID          29746 non-null  int64  
 1   SEASON_ID          29746 non-null  object 
 2   LEAGUE_ID          29746 non-null  int64  
 3   TEAM_ID            29746 non-null  int64  
 4   TEAM_ABBREVIATION  29738 non-null  object 
 5   PLAYER_AGE         29746 non-null  float64
 6   GP                 29746 non-null  int64  
 7   GS                 23264 non-null  float64
 8   MIN                28977 non-null  float64
 9   FGM                29746 non-null  int64  
 10  FGA                29746 non-null  int64  
 11  FG_PCT             29732 non-null  float64
 12  FG3M               23713 non-null  float64
 13  FG3A               23713 non-null  float64
 14  FG3_PCT            23491 non-null  float64
 15  FTM                29746 non-null  int64  
 16  FTA                297

<a name="1.-Create-Team-Matchups-and-Targets"></a>
# 1. Create Team Matchups and Targets

[Return to top](#Feature-Engineering)

<a name="1.1.-Remove-any-Orphan-Matchups"></a>
## 1.1. Remove any Orphan Matchups

[Return to top](#Feature-Engineering)

We need to first check that there are home and away team matches within `team_bs_df`.

In [6]:
# do we have sets of home and away team matches?
team_bs_df['GAME_ID'].value_counts().value_counts()

count
2    52085
1      210
3       19
Name: count, dtype: int64

In [7]:
# clean up the data
team_bs_df_cleaned = utl.clean_team_bs_data(team_bs_df)

In [8]:
# check that it worked
team_bs_df_cleaned['GAME_ID'].value_counts().value_counts()

count
2    52102
Name: count, dtype: int64

<a name="1.2.-Reshape-to-Game-Matchups"></a>
## 1.2. Reshape to Game Matchups

[Return to top](#Feature-Engineering)

In [9]:
# identify non-stats columns
non_stats_cols = ['SEASON_ID', 'GAME_ID', 'GAME_DATE', 'MATCHUP']

# reshape team box score data to wide format so each row is a game matchup
team_bs_matchups_df = utl.reshape_team_bs_to_matchups(team_bs_df_cleaned, non_stats_cols)

In [10]:
team_bs_matchups_df.head()

,SEASON_ID,HOME_TEAM_ID,HOME_TEAM_ABBREVIATION,HOME_TEAM_NAME,GAME_ID,GAME_DATE,HOME_WL,HOME_MIN,HOME_PTS,HOME_FGM,HOME_FGA,HOME_FG_PCT,HOME_FG3M,HOME_FG3A,HOME_FG3_PCT,HOME_FTM,HOME_FTA,HOME_FT_PCT,HOME_OREB,HOME_DREB,HOME_REB,HOME_AST,HOME_STL,HOME_BLK,HOME_TOV,...,HOME_PLUS_MINUS,AWAY_TEAM_ID,AWAY_TEAM_ABBREVIATION,AWAY_TEAM_NAME,AWAY_WL,AWAY_MIN,AWAY_PTS,AWAY_FGM,AWAY_FGA,AWAY_FG_PCT,AWAY_FG3M,AWAY_FG3A,AWAY_FG3_PCT,AWAY_FTM,AWAY_FTA,AWAY_FT_PCT,AWAY_OREB,AWAY_DREB,AWAY_REB,AWAY_AST,AWAY_STL,AWAY_BLK,AWAY_TOV,AWAY_PF,AWAY_PLUS_MINUS
0,21983,1610612737,ATL,Atlanta Hawks,28300014,1983-10-29,W,240,117,49,94,0.521,0,1.0,0.0,19,30,0.633,27.0,21.0,48.0,28,14.0,7,23,...,NaN,1610612765,DET,Detroit Pistons,L,240,115,40,88,0.455,3,4.0,0.75,32,38,0.842,23.0,22.0,45.0,22,10.0,2,21,29,NaN
1,21983,1610612737,ATL,Atlanta Hawks,28300027,1983-11-01,W,240,95,38,81,0.469,0,0.0,NaN,19,30,0.633,12.0,29.0,41.0,20,7.0,10,16,...,NaN,1610612764,WAS,Washington Bullets,L,240,92,35,74,0.473,0,0.0,NaN,22,34,0.647,10.0,37.0,47.0,20,5.0,3,22,26,NaN
2,21983,1610612737,ATL,Atlanta Hawks,28300041,1983-11-04,W,240,103,42,86,0.488,1,1.0,1.0,18,26,0.692,19.0,27.0,46.0,31,14.0,13,18,...,NaN,1610612741,CHI,Chicago Bulls,L,240,90,30,80,0.375,0,3.0,0.00,30,41,0.732,25.0,28.0,53.0,13,6.0,3,26,25,NaN
3,21983,1610612737,ATL,Atlanta Hawks,28300101,1983-11-15,W,240,107,45,84,0.536,0,0.0,NaN,17,21,0.810,17.0,24.0,41.0,24,7.0,10,18,...,NaN,1610612746,SDC,San Diego Clippers,L,240,102,40,78,0.513,0,2.0,0.00,22,33,0.667,17.0,22.0,39.0,28,5.0,6,19,20,NaN
4,21983,1610612737,ATL,Atlanta Hawks,28300112,1983-11-17,W,240,99,35,66,0.530,1,2.0,0.5,28,40,0.700,10.0,35.0,45.0,20,5.0,12,23,...,NaN,1610612755,PHL,Philadelphia 76ers,L,240,94,35,87,0.402,2,5.0,0.40,22,27,0.815,16.0,24.0,40.0,17,7.0,5,16,32,NaN


<a name="1.3.-Create-Target-Variables"></a>
## 1.3. Create Target Variables

[Return to top](#Feature-Engineering)

There are three targets of interest:

1. **Total Game Points (over / under):** This can be calculated as the sum of `HOME_PTS + AWAY_PTS`.
2. **Difference in Game Points (plus / minus):** This can be calculated in relation to the home team as the following difference: `HOME_PTS - AWAY_PTS`.
3. **Game Winner (moneyline):** This can be defined in relation to the home team using the `HOME_WL` column, where a win for the home team is equal to 1 and a loss for the home team equal to 0. We will create a new column called `GAME_RESULT` for this indicator.

In [11]:
# create the above three target variables
team_bs_matchups_df = utl.create_target_variables(team_bs_matchups_df, 'HOME_WL', 'HOME_PTS', 'AWAY_PTS')

In [12]:
team_bs_matchups_df[['GAME_DATE', 'GAME_ID',  'HOME_TEAM_NAME', 'AWAY_TEAM_NAME', 'HOME_PTS', 'AWAY_PTS', 'GAME_RESULT', 'TOTAL_PTS', 'PLUS_MINUS']].tail()

,GAME_DATE,GAME_ID,HOME_TEAM_NAME,AWAY_TEAM_NAME,HOME_PTS,AWAY_PTS,GAME_RESULT,TOTAL_PTS,PLUS_MINUS
52096,2024-01-06,22300494,Washington Wizards,New York Knicks,105,121,0,226,-16
52097,2024-01-08,22300508,Washington Wizards,Oklahoma City Thunder,128,136,0,264,-8
52098,2024-01-15,22300558,Washington Wizards,Detroit Pistons,117,129,0,246,-12
52099,2024-01-20,22300594,Washington Wizards,San Antonio Spurs,127,131,0,258,-4
52100,2024-01-21,22300602,Washington Wizards,Denver Nuggets,104,113,0,217,-9


<a name="2.-Create-Rolling-Window-Statistics"></a>
# 2. Create Rolling Window Statistics

[Return to top](#Feature-Engineering)

Here we create average box scores for each team over a rolling window of the previous $n$-games.

In [13]:
# identify stats columns
non_stats_cols = ['SEASON_ID', 'GAME_ID', 'GAME_DATE', 'HOME_TEAM_ID', 'AWAY_TEAM_ID',
                  'HOME_TEAM_NAME', 'AWAY_TEAM_NAME', 'HOME_WL', 'AWAY_WL',
                  'HOME_TEAM_ABBREVIATION', 'AWAY_TEAM_ABBREVIATION']
stats_cols = [col for col in team_bs_matchups_df.columns if col not in non_stats_cols]

In [14]:
# calculate rolling averages for each statistic and add them to the DataFrame
team_bs_matchups_roll_df = utl.process_rolling_stats(
    team_bs_matchups_df, 
    stats_cols, 
    window_size=20, # the number of games to include in the rolling window
    min_obs=1       # the minimum number of observations present within the window to yield an aggregate value
)

In [15]:
team_bs_matchups_roll_df.tail()

,SEASON_ID,HOME_TEAM_ID,HOME_TEAM_ABBREVIATION,HOME_TEAM_NAME,GAME_ID,GAME_DATE,HOME_WL,HOME_MIN,HOME_PTS,HOME_FGM,HOME_FGA,HOME_FG_PCT,HOME_FG3M,HOME_FG3A,HOME_FG3_PCT,HOME_FTM,HOME_FTA,HOME_FT_PCT,HOME_OREB,HOME_DREB,HOME_REB,HOME_AST,HOME_STL,HOME_BLK,HOME_TOV,...,ROLL_HOME_STL,ROLL_HOME_BLK,ROLL_HOME_TOV,ROLL_HOME_PF,ROLL_HOME_PLUS_MINUS,ROLL_AWAY_MIN,ROLL_AWAY_PTS,ROLL_AWAY_FGM,ROLL_AWAY_FGA,ROLL_AWAY_FG_PCT,ROLL_AWAY_FG3M,ROLL_AWAY_FG3A,ROLL_AWAY_FG3_PCT,ROLL_AWAY_FTM,ROLL_AWAY_FTA,ROLL_AWAY_FT_PCT,ROLL_AWAY_OREB,ROLL_AWAY_DREB,ROLL_AWAY_REB,ROLL_AWAY_AST,ROLL_AWAY_STL,ROLL_AWAY_BLK,ROLL_AWAY_TOV,ROLL_AWAY_PF,ROLL_AWAY_PLUS_MINUS
52096,22023,1610612764,WAS,Washington Wizards,22300494,2024-01-06,L,240,105,40,85,0.471,13,37.0,0.351,12,20,0.600,12.0,33.0,45.0,33,5.0,3,20,...,7.6,6.6,13.6,19.2,-4.4,240.0,120.6,42.2,89.4,0.472,13.6,35.0,0.381,22.6,27.2,0.832,10.4,34.6,45.0,24.2,7.4,4.4,13.8,21.4,4.6
52097,22023,1610612764,WAS,Washington Wizards,22300508,2024-01-08,L,240,128,44,89,0.494,15,32.0,0.469,25,29,0.862,12.0,26.0,38.0,32,7.0,5,14,...,7.2,5.6,14.2,19.6,-10.4,240.2,122.6,45.6,90.4,0.507,11.4,31.8,0.354,20.0,24.6,0.819,10.2,31.4,41.6,27.2,8.0,6.0,12.0,22.6,2.0
52098,22023,1610612764,WAS,Washington Wizards,22300558,2024-01-15,L,240,117,44,95,0.463,19,41.0,0.463,10,11,0.909,8.0,25.0,33.0,30,8.0,7,8,...,6.6,5.0,13.0,20.0,-10.4,250.0,121.2,46.0,95.2,0.482,13.6,36.2,0.381,15.6,21.6,0.729,13.6,30.2,43.8,28.0,5.8,4.0,14.8,22.4,-11.2
52099,22023,1610612764,WAS,Washington Wizards,22300594,2024-01-20,L,240,127,49,103,0.476,9,29.0,0.310,20,23,0.870,14.0,32.0,46.0,28,10.0,4,11,...,6.0,5.8,11.4,20.4,-6.8,240.2,112.4,42.8,90.4,0.475,11.2,34.4,0.330,15.6,19.8,0.776,9.0,32.0,41.0,29.4,8.2,4.8,11.2,16.6,-2.6
52100,22023,1610612764,WAS,Washington Wizards,22300602,2024-01-21,L,240,104,39,91,0.429,8,27.0,0.296,18,22,0.818,12.0,28.0,40.0,31,6.0,9,13,...,7.0,5.6,12.4,20.8,-8.8,240.4,113.2,42.2,85.2,0.496,10.8,29.8,0.362,18.0,21.8,0.821,11.4,30.8,42.2,24.4,6.6,5.4,11.2,18.2,-1.2


In [16]:
# write out the matchups with rolling features
team_bs_matchups_roll_df.to_csv('../data/processed/nba_team_matchups_rolling_stats_1984_2024_r20.csv', index=False)